# Regression simulation

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import data
import copy
from rolldecay.bis_system import BisSystem

from rolldecay import database
from mdldb.tables import Run
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
from rolldecayestimators import symbols, equations
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer


import sympy as sp

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

import signal_lab
from sqlalchemy.inspection import inspect
import seaborn as sns
import docs
import pickle
import gc
import dill
from rolldecayestimators.substitute_dynamic_symbols import lambdify,run
from rolldecayestimators.polynom_estimator import Polynom

## omega0

In [ ]:
omega_hat_polynom = Polynom.load('omega0_hat_polynom.sym')

data = pd.read_pickle('data.sav')
data['omega_hat'] = omega_hat_polynom.predict(data)
y_s = pd.read_pickle('y.sav')

## $\hat{B_1}$

In [ ]:
filename = 'B_1_hat_model.sav'
B_1_hat_model = pickle.load(open(filename, 'rb'))

In [ ]:
y = y_s['B_1_hat']
#X=data[zeta_model.keys].copy()  # Handle categorical data later
#zeta_model.score(X=X,y=y)

In [ ]:
B_1_hat_polynom = Polynom.load('B_1_hat_polynom.sym')
B_1_hat_polynom.score(X=data,y=y)

## $\hat{B_2}$

In [ ]:
filename = 'B_2_hat_model.sav'
B_2_hat_model = pickle.load(open(filename, 'rb'))

In [ ]:
B_2_hat_polynom = Polynom.load('B_2_hat_polynom.sym')
B_2_hat_polynom.score(X=data,y=y)

## Predict parameters

In [ ]:
predict = data.copy()
#predict['omega_hat'] = omega0_hat_model.predict(data[omega0_hat_model.keys])
predict['omega_hat'] = omega_hat_polynom.predict(data)

#predict['zeta'] = zeta_model.predict(data[zeta_model.keys])
predict['B_1_hat'] = B_1_hat_polynom.predict(data)

#predict['d'] = d_model.predict(data[d_model.keys])
predict['B_2_hat'] = B_2_hat_polynom.predict(data)


## Simulate

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_quadratic_b', limit_score=0.90)
db = database.get_db()

In [ ]:
df_rolldecay_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=0.0)

In [ ]:
predict.head()

In [ ]:
meta_data = predict.iloc[0]

In [ ]:
meta_data_full_scale = df_rolldecay.loc[meta_data.name]

In [ ]:
db_run = db.session.query(Run).get(int(meta_data.name))
df = database.load_run(db_run)

In [ ]:
df.plot(y='phi')

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
#scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25), phi1d_start_tolerance=0.015)
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
#    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
omega_hat_equation = equations.omega_hat_equation.subs(symbols.omega,symbols.omega0)
omega_hat_equation

In [ ]:
omega0_equation = sp.Eq(symbols.omega0,sp.solve(omega_hat_equation,symbols.omega0)[0])
omega0_equation

In [ ]:
meta_data['omega_hat']

In [ ]:
omega0_lambda = lambdify(sp.solve(omega0_equation,symbols.omega0)[0])

In [ ]:
C_1A = equations.new_symbols[symbols.C_1]
C_1A_equation = sp.Eq(C_1A,sp.solve(EstimatorQuadraticB.omgea0_equation,C_1A)[0])
C_1A_equation

In [ ]:
C_1A_lambda = lambdify(sp.solve(C_1A_equation,C_1A)[0])

In [ ]:
B_1_equation= sp.Eq(symbols.B_1,sp.solve(equations.B_1_hat_equation,symbols.B_1)[0])
B_1_equation

In [ ]:
B_2_equation= sp.Eq(symbols.B_2,sp.solve(equations.B_2_hat_equation,symbols.B_2)[0])
B_2_equation

In [ ]:
B_1_lambda=lambdify(sp.solve(B_1_equation,symbols.B_1)[0])
B_2_lambda=lambdify(sp.solve(B_2_equation,symbols.B_2)[0])

In [ ]:
equations.normalize_equations[symbols.B_1]

In [ ]:
B_1A = equations.new_symbols[symbols.B_1]
B_1A_lambda = lambdify(sp.solve(equations.normalize_equations[symbols.B_1],B_1A)[0])

B_2A = equations.new_symbols[symbols.B_2]
B_2A_lambda = lambdify(sp.solve(equations.normalize_equations[symbols.B_2],B_2A)[0])

In [ ]:
run_data = meta_data.copy()
run_data['g']=9.81
run_data['rho']=1000
run_data['m']=run_data['Disp']*run_data['rho']
run_data['omega0']=run(function=omega0_lambda, inputs=run_data)
run_data['C_1A']=run(function=C_1A_lambda, inputs=run_data)
run_data['A_44']=run(function=EstimatorQuadraticB.functions['A44'], inputs=run_data)
run_data['B_1']=run(function=B_1_lambda, inputs=run_data)
run_data['B_2']=run(function=B_2_lambda, inputs=run_data)
run_data['B_1A']=run(function=B_1A_lambda, inputs=run_data)
run_data['B_2A']=run(function=B_2A_lambda, inputs=run_data)


In [ ]:
run_data['Disp']

In [ ]:
direct_estimator = EstimatorQuadraticB.load(data=run_data, X=X)

In [ ]:
direct_estimator.plot_fit()

In [ ]:
run_data['omega0']

In [ ]:
direct_estimator.omega0

In [ ]:
direct_estimator.plot_error()

In [ ]:
direct_estimator.score()

### Cutting

In [ ]:
direct_estimator = EstimatorQuadraticB.load(data=run_data, X=X.loc[0:400])

In [ ]:
direct_estimator.plot_fit()

In [ ]:
direct_estimator.plot_error()

In [ ]:
direct_estimator.score()

### Using "real" omega0

In [ ]:
run_data['omega0']=meta_data_full_scale.omega0
run_data['C_1A']=run(function=C_1A_lambda, inputs=run_data)
run_data['A_44']=run(function=EstimatorQuadraticB.functions['A44'], inputs=run_data)
run_data['B_1']=run(function=B_1_lambda, inputs=run_data)
run_data['B_2']=run(function=B_2_lambda, inputs=run_data)
run_data['B_1A']=run(function=B_1A_lambda, inputs=run_data)
run_data['B_2A']=run(function=B_2A_lambda, inputs=run_data)

In [ ]:
direct_estimator = EstimatorQuadraticB.load(data=run_data, X=X)

In [ ]:
fig,ax=plt.subplots()
direct_estimator.plot_fit(ax=ax)

meta_data_ikeda = df_rolldecay_ikeda.loc[meta_data.name]
meta_data_ikeda['omega0']=meta_data_full_scale.omega0
ikeda_estimator = IkedaQuadraticEstimator.load(data=meta_data_ikeda, X=X)
ikeda_estimator.plot_fit(ax=ax, model_test=False)

In [ ]:
direct_estimator.score()

In [ ]:
ikeda_estimator.score()

## Do it for many...

In [ ]:
def evaluate(meta_data):
    
    if not meta_data.name in df_rolldecay.index:
        return
    
    meta_data_full_scale = df_rolldecay.loc[meta_data.name]
    db_run = db.session.query(Run).get(int(meta_data.name))
    df = database.load_run(db_run)
    
    run_data = meta_data.copy()
    run_data['omega0']=meta_data_full_scale.omega0
    run_data['g']=9.81
    run_data['rho']=1000
    run_data['m']=run_data['Disp']*run_data['rho']
    run_data['C_1A']=run(function=C_1A_lambda, inputs=run_data)
    run_data['A_44']=run(function=EstimatorQuadraticB.functions['A44'], inputs=run_data)
    run_data['B_1']=run(function=B_1_lambda, inputs=run_data)
    run_data['B_2']=run(function=B_2_lambda, inputs=run_data)
    run_data['B_1A']=run(function=B_1A_lambda, inputs=run_data)
    run_data['B_2A']=run(function=B_2A_lambda, inputs=run_data)
    
    #preprocessor['scaler'].scale_factor=db_run.model.scale_factor
    preprocessor.fit(df)
    X = preprocessor.transform(df)
    
    direct_estimator = EstimatorQuadraticB.load(data=run_data, X=X)
    fig,ax=plt.subplots()
    direct_estimator.plot_fit(ax=ax)
    
    if meta_data.name in df_rolldecay_ikeda.index:
        meta_data_ikeda = df_rolldecay_ikeda.loc[meta_data.name]
        meta_data_ikeda['omega0']=meta_data_full_scale.omega0
        ikeda_estimator = IkedaQuadraticEstimator.load(data=meta_data_ikeda, X=X)
        ikeda_estimator.plot_fit(ax=ax, model_test=False)
    

In [ ]:
for run_id,meta_data in predict.iloc[0:50].iterrows():
    evaluate(meta_data)
    gc.collect()
    